In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
import os
import cv2
from tqdm import tqdm

In [2]:
happy_folder='/content/drive/MyDrive/Datasets MLAI/Emotion Images/happy'
sad_folder='/content/drive/MyDrive/Datasets MLAI/Emotion Images/sad'

In [3]:
# Function to load images
def load_images(folder_path, label):
    images = []
    labels = []

    for filename in tqdm(os.listdir(folder_path)):
        if filename.endswith('.jpg'):
            # Read the image
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            # Append the image and label to the lists
            images.append(img)
            labels.append(label)
    return images, labels


In [4]:
# Load and preprocess happy images (label: 1)
happy_images, happy_labels = load_images(happy_folder, 1)

100%|██████████| 5054/5054 [00:00<00:00, 20052.65it/s]


In [5]:
# Load and preprocess sad images (label: 0)
sad_images, sad_labels = load_images(sad_folder, 0)

100%|██████████| 3098/3098 [00:50<00:00, 61.71it/s] 


In [6]:
# Merge the lists
all_images = happy_images + sad_images
all_labels = happy_labels + sad_labels

In [8]:
# Convert lists to numpy arrays
x = np.array(all_images)
y = np.array(all_labels)

In [9]:
# Shuffle the data (if needed)
shuffle_indices = np.arange(x.shape[0])
np.random.shuffle(shuffle_indices)

x = x[shuffle_indices]
y = y[shuffle_indices]

In [10]:
print(x.shape)
print(y.shape)

(3143, 96, 96, 3)
(3143,)


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical

In [12]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [13]:
# Convert labels to categorical by encoding
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [14]:
# Shuffle the training data
x_train, y_train = shuffle(x_train, y_train, random_state=42)

In [22]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(96, 96, 3))

In [23]:
print(base_model.input_shape)

(None, 96, 96, 3)


In [24]:
# Unfreeze some layers
for layer in base_model.layers:
    layer.trainable = False

In [25]:
# Add our own classification layers
model1 = models.Sequential()
model1.add(base_model)
model1.add(layers.Flatten())
model1.add(layers.Dense(256, activation='relu'))
model1.add(layers.Dropout(0.5))
model1.add(layers.Dense(2, activation='softmax'))  # num_classes is the number of classes in our dataset

In [26]:
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [27]:
print(x_train.shape)
print(y_train.shape)

(2828, 96, 96, 3)
(2828, 2)


In [28]:
model1.fit(x_train, y_train, epochs=100, validation_split=0.2 )


Epoch 1/100
71/71 [==============================] - 11s 59ms/step - loss: 2.5764 - accuracy: 0.9691 - val_loss: 0.3914 - val_accuracy: 0.9859
Epoch 2/100
71/71 [==============================] - 3s 36ms/step - loss: 0.2812 - accuracy: 0.9779 - val_loss: 0.2935 - val_accuracy: 0.9859
Epoch 3/100
71/71 [==============================] - 2s 35ms/step - loss: 0.0856 - accuracy: 0.9859 - val_loss: 0.2028 - val_accuracy: 0.9841
Epoch 4/100
71/71 [==============================] - 2s 34ms/step - loss: 0.1040 - accuracy: 0.9836 - val_loss: 0.2622 - val_accuracy: 0.9859
Epoch 5/100
71/71 [==============================] - 2s 34ms/step - loss: 0.0878 - accuracy: 0.9876 - val_loss: 0.1672 - val_accuracy: 0.9841
Epoch 6/100
71/71 [==============================] - 2s 34ms/step - loss: 0.0359 - accuracy: 0.9907 - val_loss: 0.2198 - val_accuracy: 0.9859
Epoch 7/100
71/71 [==============================] - 2s 34ms/step - loss: 0.0269 - accuracy: 0.9925 - val_loss: 0.2255 - val_accuracy: 0.9859
Epoch

In [31]:
# Save model1 architecture to a JSON file
model1_json = model1.to_json()
with open('model1_architecture.json', 'w') as json_file:
    json_file.write(model1_json)

# Save model1 weights to a HDF5 file
model1.save_weights('model1_weights.h5')

In [29]:
# Evaluate the model on the test set
test_loss, test_acc = model1.evaluate(x_test, y_test)
print(f'Test Accuracy: {test_acc}')

10/10 [==============================] - 2s 193ms/step - loss: 0.0933 - accuracy: 0.9968
Test Accuracy: 0.9968253970146179
